In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

# Для DJ

In [ ]:
#Cкачиваем перечень бумаг, входящих в индекс
path = 'https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI'
dji = pd.read_html(path)[0][['Symbol', 'Company Name']]
dji_new = dji.set_index('Symbol')
ticker = dji['Symbol'].values
ticker

In [ ]:
#Задаем период
start_time = datetime.datetime(2018,1,1)
finish_time = datetime.datetime(2018,12,31)

In [ ]:
#Загружаем цену закрытия при помощи pandas_datareader
#Цена закрытия - наиболее актуальная оценка ценной бумаги
#Можно также было выкачать Adj Close, она учитывает выплаты дивидендов, но результаты очень похожие
#Можно учитывать не цену,а доходность как темпы прироста скорректированных цен
#Но там граф странный получался, по нему ценные бумаги мало похожи друг на друга 

web_archiv = {}
for i in ticker:
    web_archiv[i] = web.DataReader(i, 'yahoo', start_time, finish_time)['Close']
    
web_df = pd.DataFrame(web_archiv)
web_df.head()

In [ ]:
#Проверка, что нет NA
web_df.info()

In [ ]:
#Построение взаимно-корреляционной матрицы
#В качестве порога выберем 0.8
corr = web_df.corr()
corr = corr.replace(1,0)
graph = (abs(corr)>0.8).astype(int)

In [ ]:
#Строим и визуализируем граф
m = graph.values
n = ticker
G = nx.from_numpy_matrix(m)
nodes = dict(zip(G.nodes(), n))
G1 = nx.relabel_nodes(G, nodes)
nx.draw_networkx(G1, pos = nx.kamada_kawai_layout(G1))
plt.show()

In [ ]:
#Находим клику 
#Длина максимальной клики 4, но есть еще клики такой длины
cliques = list(nx.find_cliques(G1))
code = max(cliques, key=len)
code

In [ ]:
#Функция, чтобы выбрать все максимальные клики 
def by_size(words,size):
    result = []
    for word in words:
        if len(word)==size:
            result.append(word)
    return result

In [ ]:
#Максимальные клики
max_cliques = by_size(cliques, len(code))
max_cliques

In [ ]:
#Вычислим кумулятивную процентную доходность 
daily_pct = (web_df/web_df.shift(1)-1).fillna(0)
daily_sum=(daily_pct*100).cumsum()
daily_sum.head(5)

In [ ]:
#Визуалириуем кумулятивную процентную доходность для составляющих максимальных клик
for i in range(len(max_cliques)):
    daily_sum[max_cliques[i]].plot()
    plt.title('Кумулятивная процентная доходность для клики' + '\n' + str(max_cliques[i]))
    plt.show()

In [ ]:
#Визуалириуем кумулятивную процентную доходность для максимальных клик
for i in range(len(max_cliques)):
    daily_sum[max_cliques[i]].sum(axis=1).plot()
    plt.title('Кумулятивная процентная доходность для клики' + '\n' + str(max_cliques[i]))
    plt.show()

In [ ]:
#Визуализация риска для максимальных клик

for i in range(len(max_cliques)):
    cli = []
    t = i +1
    cli = daily_pct[max_cliques[i]]
    cli['risk'] = cli.mean(numeric_only=True, axis=1)/cli.std(numeric_only=True, axis=1)
    cli['risk'].plot()
    plt.title('Риск для клики %i' %t + '\n' + str(max_cliques[i]))
    plt.show()

In [ ]:
#Разброс акций относительно их мат. ошидания и СКО

df_risk=pd.concat([daily_pct.mean(),daily_pct.std()],axis=1)
df_risk.columns=["mean",'std']
sns.scatterplot(x="std", y="mean",data=df_risk,hue=df_risk.index)

In [ ]:
#Нахождение бумаг на ПО границе

max_mean = df_risk.loc[df_risk['mean'] == df_risk['mean'].max()]
min_std = df_risk.loc[df_risk['std'] == df_risk['std'].min()]

sns.scatterplot(x="std", y="mean",data=df_risk)
#акция с максимальной доходностью
plt.scatter(x=max_mean['std'], y=max_mean['mean'], c='red', marker='D')
plt.text(x=max_mean['std'], y=max_mean['mean'], s=list(max_mean.index))
#акция с минимальным риском
plt.scatter(x=min_std['std'], y=min_std['mean'], c='blue', marker='D')
plt.text(x=min_std['std'], y=min_std['mean'], s=list(min_std.index))
plt.title('Поиск акций на эффективной границе')
plt.show()

In [ ]:
print("Компания, у которой акции имеют минимальный риск:", dji_new.loc[min_std.index]['Company Name'].to_string(index=False))
print("Компания, у которой акции имеют максимальную доходность:", dji_new.loc[max_mean.index]['Company Name'].to_string(index=False))

# Для S&P 500

In [ ]:
#Cкачиваем перечень бумаг, входящих в индекс
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][1:]
sp = data[data.columns[:2]]
sp = sp.rename(columns={0: "Company Name", 1: "Symbol"})
sp_new = sp.set_index('Symbol')
sp.head()

In [ ]:
ticker_sp = sp['Symbol'].values
ticker_sp

In [ ]:
web_archiv_sp = {}
for i in ticker_sp:
    web_archiv_sp[i] = web.DataReader(i, 'yahoo', start_time, finish_time)['Close']

In [ ]:
sp_df = pd.DataFrame(web_archiv_sp)
sp_df.head()

In [ ]:
#проверка, что нет NA
sp_df.info()

In [ ]:
#Построение взаимно-корреляционной матрицы
#В качестве порога выберем 0.8

corr_sp = sp_df.corr()
corr_sp = corr_sp.replace(1,0)
graph_sp = (abs(corr_sp)>0.8).astype(int)

In [ ]:
#Строим и визуализируем граф
m_sp = graph_sp.values
n_sp = ticker_sp
G_sp = nx.from_numpy_matrix(m_sp)
nodes_sp = dict(zip(G_sp.nodes(), n_sp))
G1_sp = nx.relabel_nodes(G_sp, nodes_sp)
nx.draw_networkx(G1_sp, pos = nx.kamada_kawai_layout(G1_sp))
plt.show()

In [ ]:
#Находим клику 
#Длина максимальной клики 35, но есть еще клики такой длины

cliques_sp = list(nx.find_cliques(G1_sp))
code_sp = max(cliques_sp, key=len)
print(len(code_sp))
print(code_sp)

In [ ]:
#Находим все максимальные клики, их 5
max_cliques_sp = by_size(cliques_sp, len(code_sp))
max_cliques_sp

In [ ]:
#Вычислим кумулятивную процентную доходность 
daily_pct_sp = (sp_df/sp_df.shift(1)-1).fillna(0)
daily_sum_sp=(daily_pct_sp*100).cumsum()
daily_sum_sp.head()

In [ ]:
#Визуалириуем кумулятивную процентную доходность для составляющих максимальных клик
#У всех максимальных клик кумулятивная доходность падает - это может насторожить при составлении портфеля из таких акций
for i in range(len(max_cliques_sp)):
    t = i+1
    daily_sum_sp[max_cliques_sp[i]].plot()
    plt.title('Кумулятивная процентная доходность для клики %i' %t)
    plt.show()

In [ ]:
#Попробуем взять не цены закрытия, а процентные изменения для цен закрытия

corr_sp = daily_pct_sp.corr()
corr_sp = corr_sp.replace(1,0)
graph_sp = (abs(corr_sp)>0.8).astype(int)
m_sp = graph_sp.values
n_sp = ticker_sp
G_sp = nx.from_numpy_matrix(m_sp)
nodes_sp = dict(zip(G_sp.nodes(), n_sp))
G1_sp = nx.relabel_nodes(G_sp, nodes_sp)
nx.draw_networkx(G1_sp, pos = nx.kamada_kawai_layout(G1_sp))
plt.show()

In [ ]:
#Найдем клики, длина максимальной клики 11
cliques_sp = list(nx.find_cliques(G1_sp))
code_sp = max(cliques_sp, key=len)
print(len(code_sp))
print(code_sp)

In [ ]:
#Максимальных клик с такой длиной 2
max_cliques_sp = by_size(cliques_sp, len(code_sp))
max_cliques_sp

In [ ]:
#Визуализируем кумулятивную процентную доходность для составляющих максимальных клик
for i in range(len(max_cliques_sp)):
    t = i+1
    daily_sum_sp[max_cliques_sp[i]].plot()
    plt.title('Кумулятивная процентная доходность для клики %i' %t)
    plt.show()

In [ ]:
#Визуалириуем кумулятивную процентную доходность для максимальных клик
for i in range(len(max_cliques_sp)):
    t = i+1
    daily_sum_sp[max_cliques_sp[i]].sum(axis=1).plot()
    plt.title('Кумулятивная процентная доходность для клики %i' %t)
    plt.show()

In [ ]:
#Визуализация риска для максимальных клик

for i in range(len(max_cliques_sp)):
    cli = []
    t = i+1
    cli = daily_pct_sp[max_cliques_sp[i]]
    cli['risk'] = cli.mean(numeric_only=True, axis=1)/cli.std(numeric_only=True, axis=1)
    cli['risk'].plot()
    plt.title('Риск для клики %i' %t)
            #  + '\n' + str(max_cliques_sp[i]))
    plt.show()

In [ ]:
#Разброс акций относительно их мат. ошидания и СКО

df_risk_sp=pd.concat([daily_pct_sp.mean(),daily_pct_sp.std()],axis=1)
df_risk_sp.columns=["mean",'std']
sns.scatterplot(x="std", y="mean",data=df_risk_sp,hue=df_risk_sp.index, legend = False)

In [ ]:
#Нахождение бумаг на ПО границе

max_mean_sp = df_risk_sp.loc[df_risk_sp['mean'] == df_risk_sp['mean'].max()]
min_std_sp = df_risk_sp.loc[df_risk_sp['std'] == df_risk_sp['std'].min()]

sns.scatterplot(x="std", y="mean",data=df_risk_sp)
#акция с максимальной доходностью
plt.scatter(x=max_mean_sp['std'], y=max_mean_sp['mean'], c='red', marker='D')
plt.text(x=max_mean_sp['std'], y=max_mean_sp['mean']+0.0001, s=list(max_mean_sp.index))
#акция с минимальным риском
plt.scatter(x=min_std_sp['std'], y=min_std_sp['mean'], c='blue', marker='D')
plt.text(x=min_std_sp['std']-0.001, y=min_std_sp['mean']+0.0005, s=list(min_std_sp.index))
plt.title('Поиск акций на эффективной границе')
plt.show()

In [ ]:
print("Компания, у которой акции имеют минимальный риск:", sp_new.loc[min_std_sp.index]['Company Name'].to_string(index=False))
print("Компания, у которой акции имеют максимальную доходность:", sp_new.loc[max_mean_sp.index]['Company Name'].to_string(index=False))